# classifierchain でラベル相関学習を強化する

In [1]:
import gc
import re
import math
import pickle
import joblib
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier

warnings.simplefilter('ignore')

In [2]:
import os
import random as rn
import numpy as np


def set_seed(seed=0):
    os.environ["PYTHONHASHSEED"] = str(seed)

    rn.seed(seed)
    np.random.seed(seed)

In [3]:
from sklearn.metrics import log_loss


def score(Y, Y_pred):
    _, n_classes = Y.shape

    losses = []

    for j in range(n_classes):
        loss = log_loss(Y.iloc[:, j], Y_pred.iloc[:, j], labels=[0, 1])

        losses.append(loss)

    return np.mean(losses)

In [4]:
import sys
sys.path.append('../input/iterativestratification')

import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection._split import _BaseKFold


class MultilabelGroupStratifiedKFold(_BaseKFold):
    def __init__(self, n_splits=5, random_state=None, shuffle=False):
        super().__init__(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

    def _iter_test_indices(self, X=None, y=None, groups=None):
        cv = MultilabelStratifiedKFold(
            n_splits=self.n_splits,
            random_state=self.random_state,
            shuffle=self.shuffle,
        )

        value_counts = groups.value_counts()
        regular_index = value_counts.loc[
            (value_counts == 6) | (value_counts == 12) | (value_counts == 18)
        ].index.sort_values()
        irregular_index = value_counts.loc[
            (value_counts != 6) & (value_counts != 12) & (value_counts != 18)
        ].index.sort_values()

        group_to_fold = {}
        tmp = Y.groupby(groups).mean().loc[regular_index]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            group_to_fold.update({group: fold for group in tmp.index[test]})

        sample_to_fold = {}
        tmp = Y.loc[groups.isin(irregular_index)]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            sample_to_fold.update({sample: fold for sample in tmp.index[test]})

        folds = groups.map(group_to_fold)
        is_na = folds.isna()
        folds[is_na] = folds[is_na].index.map(sample_to_fold).values

        for i in range(self.n_splits):
            yield np.where(folds == i)[0]

In [5]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin


class ClippedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, copy=True, high=0.99, low=0.01):
        self.copy = copy
        self.high = high
        self.low = low

    def fit(self, X, y=None):
        self.data_max_ = X.quantile(q=self.high)
        self.data_min_ = X.quantile(q=self.low)

        return self

    def transform(self, X):
        if self.copy:
            X = X.copy()

        X.clip(self.data_min_, self.data_max_, axis=1, inplace=True)

        return X

In [6]:
import pandas as pd


def compute_row_statistics(X, prefix=""):
    Xt = pd.DataFrame()

    for agg_func in [
        # "min",
        # "max",
        "mean",
        "std",
        "kurtosis",
        "skew",
    ]:
        Xt[f"{prefix}{agg_func}"] = X.agg(agg_func, axis=1)

    return Xt

In [7]:
#dtype = {"cp_type": "category", "cp_dose": "category"}
#index_col = "sig_id"
#
#train_features = pd.read_csv(
#    "../input/lish-moa/train_features.csv", dtype=dtype, index_col=index_col
#)
#X = train_features.select_dtypes("number")
#Y_nonscored = pd.read_csv(
#    "../input/lish-moa/train_targets_nonscored.csv", index_col=index_col
#)
#Y = pd.read_csv("../input/lish-moa/train_targets_scored.csv", index_col=index_col)
#groups = pd.read_csv(
#    "../input/lish-moa/train_drug.csv", index_col=index_col, squeeze=True
#)
#
#columns = Y.columns

In [8]:
dtype = {"cp_type": "category", "cp_dose": "category"}
index_col = "sig_id"

sys.path.append(
    r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\code"
)
import datasets

DATADIR = datasets.DATADIR

train_features = pd.read_csv(
    f"{DATADIR}/train_features.csv", dtype=dtype, index_col=index_col
)
X = train_features.select_dtypes("number")
Y_nonscored = pd.read_csv(
    f"{DATADIR}/train_targets_nonscored.csv", index_col=index_col
)
Y = pd.read_csv(f"{DATADIR}/train_targets_scored.csv", index_col=index_col)
groups = pd.read_csv(
    f"{DATADIR}/train_drug.csv", dtype=dtype, index_col=index_col, squeeze=True
)

columns = Y.columns

In [9]:
clipped_features = ClippedFeatures()
X = clipped_features.fit_transform(X)

with open("clipped_features.pkl", "wb") as f:
    pickle.dump(clipped_features, f)
# アンサンブルのために統計値, nonscoredは入れない 
#c_prefix = "c-"
#g_prefix = "g-"
#c_columns = X.columns.str.startswith(c_prefix)
#g_columns = X.columns.str.startswith(g_prefix)
#X_c = compute_row_statistics(X.loc[:, c_columns], prefix=c_prefix)
#X_g = compute_row_statistics(X.loc[:, g_columns], prefix=g_prefix)
#X = pd.concat([X, X_c, X_g], axis=1)
#
#Y_nonscored = Y_nonscored.loc[:, Y_nonscored.sum(axis=0) > 0]
#Y = pd.concat([Y, Y_nonscored], axis=1)

# objective

In [10]:
def train_and_evaluate(params):
    counts = np.empty((n_seeds * n_splits, n_classes))

    Y_pred = np.zeros((train_size, n_classes))
    Y_pred = pd.DataFrame(Y_pred, columns=Y.columns, index=Y.index)

    for i in range(n_seeds):
        set_seed(seed=i)

        if is_drug_cv:
            cv = MultilabelGroupStratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
            cv_split = cv.split(X, Y, groups)
        else:
            MultilabelStratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
            cv_split = cv.split(X, Y)

        order = rn.sample(range(Y.shape[1]), k=Y.shape[1])
        print(f"order: {order}")

        for j, (trn_idx, val_idx) in tqdm(enumerate(cv_split)):
            counts[i * n_splits + j] = Y.iloc[trn_idx].sum()

            X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
            Y_train, Y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            
            # Label Smoothing. https://www.kaggle.com/gogo827jz/self-stacking-groupcv-xgboost
            Y_train = Y_train * (1 - LBS) + 0.5 * LBS

            clf = ClassifierChain(LGBMClassifier(**params), order=order)
            clf.fit(X_train, Y_train)
            val_preds = clf.predict_proba(X_val) # list of preds per class

            Y_pred.iloc[val_idx] += val_preds / n_seeds

            joblib.dump(clf, f"model_seed_{i}_fold_{j}.jlb", compress=True)

    Y_pred[train_features["cp_type"] == "ctl_vehicle"] = 0.0

    with open("counts.pkl", "wb") as f:
        pickle.dump(counts, f)

    with open("Y_pred.pkl", "wb") as f:
        pickle.dump(Y_pred[columns], f)
        
    oof = score(Y[columns], Y_pred[columns])
    
    return oof, Y_pred

In [11]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "learning_rate": 0.1,
    }
    params["max_depth"] = trial.suggest_int("max_depth", 1, 3)
    params["num_leaves"] = trial.suggest_int("num_leaves", 2, 4)
    params["min_data_in_leaf"] = trial.suggest_int(
        "min_data_in_leaf", 
        1, 
        max(1, int(X.shape[0] * ((n_splits - 1) / n_splits) / params["num_leaves"])),
    )
    params["feature_fraction"] = trial.suggest_discrete_uniform("feature_fraction", 0.1, 1.0, 0.05)
    params["lambda_l1"] = trial.suggest_loguniform("lambda_l1", 1e-09, 10.0)
    params["lambda_l2"] = trial.suggest_loguniform("lambda_l2", 1e-09, 10.0)

    if DEBUG:
        params["n_estimators"] = 2
    #else:
    #    params["n_estimators"] = 1000
    
    oof, _ = train_and_evaluate(params)
    
    return oof

In [12]:
is_drug_cv = True
n_splits = 5
n_seeds = 1
# LBS = 0.0008  # ラベルスムージングは全然効かないからやめる
LBS = 0.0

n_trials = 50
#params = {
#    "num_leaves": 2,
#    "max_depth": 1,
#    "min_data_in_leaf": 969,
#    "objective": "binary",
#    "learning_rate": 0.01,
#}

#DEBUG = True
DEBUG = False
if DEBUG:
    columns = [
        "atp-sensitive_potassium_channel_antagonist",  # 陽性ラベル1個だけ
        "erbb2_inhibitor",  # 陽性ラベル1個だけ
        "antiarrhythmic",  # 陽性ラベル6個だけ
        "aldehyde_dehydrogenase_inhibitor",  # 陽性ラベル7個だけ
        "lipase_inhibitor",  # 陽性ラベル12個だけ
        "sphingosine_receptor_agonist",  # 陽性ラベル25個だけ
        "igf-1_inhibitor",  # 陽性ラベル37個だけ
        "potassium_channel_activator",  # 陽性ラベル55個だけ
        "potassium_channel_antagonist",  # 陽性ラベル98個だけ
        "dopamine_receptor_agonist",  # 陽性ラベル121個だけ
        "nfkb_inhibitor",  # 陽性ラベル832個
        "cyclooxygenase_inhibitor",  # 陽性ラベル435個
        "dna_inhibitor",  # 陽性ラベル402個
        "glutamate_receptor_antagonist",  # 陽性ラベル367個
        "tubulin_inhibitor",  # 陽性ラベル316個
        "pdgfr_inhibitor",  # 陽性ラベル297個
        "calcium_channel_blocker",  # 陽性ラベル281個
        "flt3_inhibitor",  # 陽性ラベル279個
        "progesterone_receptor_agonist",  # 陽性ラベル119個
        "hdac_inhibitor",  # 陽性ラベル106個
    ]
    Y = Y[columns]

    non_columns = [
        "abc_transporter_expression_enhancer",  # nonscored class
        "abl_inhibitor",  # nonscored class
    ]
    Y_nonscored = Y_nonscored[non_columns]
    
    Y = pd.concat([Y, Y_nonscored], axis=1)
    
    n_splits = 2
    n_trials = 3
    print(f"DEBUG: {DEBUG}")

In [13]:
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

In [14]:
# 確認
Y.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000779bfc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000a6266a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_0015fd391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_001626bd3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
%%time

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")

[I 2020-11-17 18:04:45,224] Using an existing study with name 'study' instead of creating a new one.
0it [00:00, ?it/s]

order: [98, 194, 107, 10, 66, 130, 124, 103, 77, 122, 91, 149, 55, 129, 35, 72, 191, 24, 158, 64, 136, 180, 154, 37, 79, 25, 18, 175, 84, 120, 143, 184, 90, 111, 80, 156, 163, 52, 141, 196, 113, 133, 201, 15, 140, 3, 23, 102, 0, 170, 126, 85, 62, 83, 16, 48, 145, 56, 61, 36, 139, 114, 159, 20, 81, 200, 125, 27, 197, 74, 31, 183, 168, 73, 165, 178, 205, 167, 147, 182, 161, 188, 105, 204, 4, 78, 177, 33, 60, 8, 11, 86, 96, 151, 19, 121, 203, 202, 89, 106, 69, 87, 50, 173, 67, 189, 169, 30, 138, 144, 75, 53, 185, 100, 57, 63, 119, 82, 45, 108, 41, 176, 14, 153, 95, 171, 42, 199, 135, 2, 34, 152, 28, 47, 21, 181, 54, 7, 12, 179, 132, 5, 9, 1, 68, 40, 101, 38, 94, 146, 127, 174, 186, 166, 193, 76, 58, 88, 59, 115, 104, 51, 192, 13, 160, 150, 128, 97, 6, 112, 110, 93, 92, 172, 22, 70, 195, 117, 148, 39, 65, 32, 134, 137, 190, 155, 116, 46, 162, 29, 142, 71, 157, 26, 131, 198, 99, 49, 118, 43, 44, 164, 123, 17, 109, 187]
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 

1it [09:33, 573.62s/it]

[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] min_data_in_leaf is set=1367, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1367
[LightGBM] [Warning] lambda_l1 is set=3.6056855520901, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.6056855520901
[LightGBM] [Warning] lambda_l2 is set=0.2777268104182044, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2777268104182044


2it [20:00, 589.69s/it]

...

[LightGBM] [Warning] feature_fraction is set=0.15000000000000002, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.15000000000000002
[LightGBM] [Warning] min_data_in_leaf is set=26, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=26
[LightGBM] [Warning] lambda_l1 is set=0.0048276879746142675, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0048276879746142675
[LightGBM] [Warning] lambda_l2 is set=1.1201862562202543, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1201862562202543


4it [20:38, 309.66s/it]

[LightGBM] [Warning] feature_fraction is set=0.15000000000000002, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.15000000000000002
[LightGBM] [Warning] min_data_in_leaf is set=26, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=26
[LightGBM] [Warning] lambda_l1 is set=0.0048276879746142675, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0048276879746142675
[LightGBM] [Warning] lambda_l2 is set=1.1201862562202543, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1201862562202543


5it [25:43, 308.78s/it]
[I 2020-11-18 22:32:11,918] Trial 93 finished with value: 0.017377247729824605 and parameters: {'max_depth': 3, 'num_leaves': 3, 'min_data_in_leaf': 26, 'feature_fraction': 0.15000000000000002, 'lambda_l1': 0.0048276879746142675, 'lambda_l2': 1.1201862562202543}. Best is trial 50 with value: 0.016896081120940355.



study.best_params:
{'feature_fraction': 0.30000000000000004, 'lambda_l1': 5.191394900888072, 'lambda_l2': 2.0588068479966517, 'max_depth': 3, 'min_data_in_leaf': 373, 'num_leaves': 3}
Wall time: 1d 4h 27min 26s


In [ ]:
params = study.best_params
params = {
    "objective": "binary",
    "learning_rate": 0.1,
    #"learning_rate": 0.01,
    #"n_estimators": 1000,
}
# params["n_estimators"] = 100  # default param

oof, Y_pred = train_and_evaluate(params)
print(oof)

0it [00:00, ?it/s]

order: [98, 194, 107, 10, 66, 130, 124, 103, 77, 122, 91, 149, 55, 129, 35, 72, 191, 24, 158, 64, 136, 180, 154, 37, 79, 25, 18, 175, 84, 120, 143, 184, 90, 111, 80, 156, 163, 52, 141, 196, 113, 133, 201, 15, 140, 3, 23, 102, 0, 170, 126, 85, 62, 83, 16, 48, 145, 56, 61, 36, 139, 114, 159, 20, 81, 200, 125, 27, 197, 74, 31, 183, 168, 73, 165, 178, 205, 167, 147, 182, 161, 188, 105, 204, 4, 78, 177, 33, 60, 8, 11, 86, 96, 151, 19, 121, 203, 202, 89, 106, 69, 87, 50, 173, 67, 189, 169, 30, 138, 144, 75, 53, 185, 100, 57, 63, 119, 82, 45, 108, 41, 176, 14, 153, 95, 171, 42, 199, 135, 2, 34, 152, 28, 47, 21, 181, 54, 7, 12, 179, 132, 5, 9, 1, 68, 40, 101, 38, 94, 146, 127, 174, 186, 166, 193, 76, 58, 88, 59, 115, 104, 51, 192, 13, 160, 150, 128, 97, 6, 112, 110, 93, 92, 172, 22, 70, 195, 117, 148, 39, 65, 32, 134, 137, 190, 155, 116, 46, 162, 29, 142, 71, 157, 26, 131, 198, 99, 49, 118, 43, 44, 164, 123, 17, 109, 187]


2it [35:00, 1046.49s/it]

# Platt Scaling
Train a Logistic Regression model to calibrate the results
- https://www.kaggle.com/gogo827jz/kernel-logistic-regression-one-for-206-targets

In [ ]:
# predict_probaでだしたY_predをロジスティク回帰で確率に補正する
# （Sigmoid関数にフィットさせ、そのSigmoid関数に通した値をCalibrationした値とする）

counts = np.empty((n_classes))

X_new = Y_pred.values
Y_cali = Y_pred.copy()

for tar in tqdm(range(Y.shape[1])):

    targets = Y.values[:, tar]
    X_targets = X_new[:, tar]
    counts[tar] = targets.sum()

    if targets.sum() >= n_splits:

        Y_cali[Y.columns[tar]] = np.zeros((Y_cali.shape[0], ))
        
        skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

        for n, (tr, te) in enumerate(skf.split(targets, targets)):
            x_tr, x_val = X_targets[tr].reshape(-1, 1), X_targets[te].reshape(-1, 1)
            y_tr, y_val = targets[tr], targets[te]

            model = LogisticRegression(penalty="none", max_iter=1000)
            model.fit(x_tr, y_tr)
            Y_cali[Y.columns[tar]].iloc[te] += model.predict_proba(x_val)[:, 1]
            
            joblib.dump(model, f"calibrate_model_target_{Y.columns[tar]}.jlb", compress=True)
        
with open("counts_calibrate.pkl", "wb") as f:
    pickle.dump(counts, f)

with open("Y_pred_calibrate.pkl", "wb") as f:
    pickle.dump(Y_cali[columns], f)
    

In [ ]:
score(Y[columns], Y_cali[columns])

# pkl check

In [ ]:
path = r"counts.pkl"
with open(path, 'rb') as f:
    counts = pickle.load(f)
counts

In [ ]:
path = r"counts_calibrate.pkl"
with open(path, 'rb') as f:
    counts = pickle.load(f)
counts

In [ ]:
path = r"Y_pred.pkl"
with open(path, 'rb') as f:
    Y_pred = pickle.load(f)
Y_pred

In [ ]:
path = r"Y_pred_calibrate.pkl"
with open(path, 'rb') as f:
    Y_pred = pickle.load(f)
Y_pred

# predict test

In [ ]:
import glob
import pathlib


test_features = pd.read_csv(
    # "../input/lish-moa/test_features.csv", dtype=dtype, index_col=index_col
    f"{DATADIR}/test_features.csv",
    dtype=dtype,
    index_col=index_col,
)
X_test = test_features.select_dtypes("number")

with open("./clipped_features.pkl", "rb") as f:
    clipped_features = pickle.load(f)
X_test = clipped_features.transform(X_test)
# アンサンブルのために統計値, nonscoredは入れない
# X_c = compute_row_statistics(X_test.loc[:, c_columns], prefix=c_prefix)
# X_g = compute_row_statistics(X_test.loc[:, g_columns], prefix=g_prefix)
# X_test = pd.concat([X_test, X_c, X_g], axis=1)

# lgbで予測
Y_test_pred = np.zeros((X_test.shape[0], len(columns)))
Y_test_pred = pd.DataFrame(Y_test_pred, columns=columns, index=test_features.index)
for i in range(n_seeds):
    set_seed(seed=i)

    for j in range(n_splits):
        clf = joblib.load(f"model_seed_{i}_fold_{j}.jlb")
        Y_test_pred += clf.predict_proba(X_test)[:, : len(columns)] / (
            n_seeds * n_splits
        )

print(Y_test_pred.shape)
display(Y_test_pred)

# lgbの予測値補正
model_paths = glob.glob(f"./calibrate_model_target_*.jlb")
for model_path in model_paths:
    target = str(pathlib.Path(model_path).stem).replace("calibrate_model_target_", "")

    if target in columns:
        # print(target)
        model = joblib.load(model_path)
        X_targets = Y_test_pred.loc[:, target].values.reshape(-1, 1)
        Y_test_pred.loc[:, target] = model.predict_proba(X_targets)[:, 1]

print(Y_test_pred.shape)
display(Y_test_pred)